# Context managers

Python provides a convenient (and safer) way to clean up resources. 

* If you open a file, you should close it when you are done, if even there was an error while working with the file. 
* If you acquire a lock, you should release it appropriately (otherwise you may have deadlock issues).
* If you open a database connection, or tcp/ip connection or any other connection or session, you should close it properly to releave the server of unnecessary load.


The most popular use of context managers is for working with files:

In [1]:
with open('../../postcell.conf') as file:
    print(file.read(500))

{
    "class_id": "2024_quarter1_saturday_advancedpythonfords",
    "instructor_id": "doJIH2jibYWOOJyGm6zsDNe93722",
    "should_send_to_server": "true",
    "student_id": "Shahbaz_Chaudhary",
    "url": "https://postcell.io/post_cell"
}


A naive translation of the above code to more common Python would be the following:

In [2]:
file = open('../../postcell.conf')
print(file.read(500))
file.close()

{
    "class_id": "2024_quarter1_saturday_advancedpythonfords",
    "instructor_id": "doJIH2jibYWOOJyGm6zsDNe93722",
    "should_send_to_server": "true",
    "student_id": "Shahbaz_Chaudhary",
    "url": "https://postcell.io/post_cell"
}


However, this doesn't quite capture the benefit the `with` clause is providing. A more accurate translation is this:

In [3]:
try:
    file = open('../../postcell.conf')
    print(file.read(500))
finally:
    file.close()

{
    "class_id": "2024_quarter1_saturday_advancedpythonfords",
    "instructor_id": "doJIH2jibYWOOJyGm6zsDNe93722",
    "should_send_to_server": "true",
    "student_id": "Shahbaz_Chaudhary",
    "url": "https://postcell.io/post_cell"
}


The first example doesn't handle the scenario where the `file.read` command causes an exception. Noice that an error with `read` will cause the `.close()` method to be skipped.

However, the second example handles such errors correctly. The file will be closed, even if `read` causes an error. (technically `open` can also cause an exception, which requires a bit more care to handle)

### Custom context managers
The implementation of context managers can be surprisingly smiple. A class just needs to implement the `__enter__` and `__exit__` functions. Here is a context manager so simple that it is silly:

In [4]:
class ExampleCtxMgr():
    
    def __enter__(self):
        print("Entering context manager")
        return "This is the return value"
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print("Exiting context manager")
        print(f"exc_type:{exc_type}, exc_value:{exc_value}, exc_traceback:{exc_traceback}")

In [5]:
with ExampleCtxMgr():
    print("yo")

Entering context manager
yo
Exiting context manager
exc_type:None, exc_value:None, exc_traceback:None


In [6]:
with ExampleCtxMgr() as msg:
    print(msg)
    print("yo")

Entering context manager
This is the return value
yo
Exiting context manager
exc_type:None, exc_value:None, exc_traceback:None


#### Exception handling in `with` clauses

Notice the values of `exc_type`, `exc_value` and `exc_traceback` in this example

In [7]:
class ExampleCtxMgrExc():
    
    def __enter__(self):
        print("Entering context manager")
        return "This is the return value"
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print("Exiting context manager")
        print(f"exc_type:{exc_type}, exc_value:{exc_value}, exc_traceback:{exc_traceback}")

In [8]:
with ExampleCtxMgrExc() as msg:
    print(msg)
    print("yo")
    raise Exception("Oops") # Something terrible happened
    print("won't get this far")

Entering context manager
This is the return value
yo
Exiting context manager
exc_type:<class 'Exception'>, exc_value:Oops, exc_traceback:<traceback object at 0x000001D516B80700>


Exception: Oops

### Function based context managers
Function based context managers provide the same functionality, but a bit easier to understand by using _decorators_ and _generators_

In [9]:
from contextlib import contextmanager

In [10]:
@contextmanager
def ExampleCtxMgrFunction():
    print("Entering context manager")
    yield "This is the return value"
    print("Exiting context manager")

In [12]:
with ExampleCtxMgrFunction() as ctx_mgr:
    print(ctx_mgr)

Entering context manager
This is the return value
Exiting context manager


### Other uses of context managers

Popular library for Bayesian models uses the context manager in a unique way:

```python
from pymc import HalfCauchy, Model, Normal, sample

with Model() as model:  # model specifications in PyMC are wrapped in a with-statement
    # Define priors
    sigma = HalfCauchy("sigma", beta=10)
    intercept = Normal("Intercept", 0, sigma=20)
    slope = Normal("slope", 0, sigma=20)

    # Define likelihood
    likelihood = Normal("y", mu=intercept + slope * x, sigma=sigma, observed=y)

    # Inference!
    # draw 3000 posterior samples using NUTS sampling
    idata = sample(3000)

```

Here, `Model` is not a typical resource, rather a true "context" where various distributions know that they are being defined as part of a specific model definition. See https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/GLM_linear.html#glm-linear for details.

See https://realpython.com/python-with-statement/#creating-custom-context-managers for more details. Some examples in this lecture adapted from that page.